<H1> Разработка инструмента для прогнозирования популярности постов в социальных сетях с применением методов машинного обучения </H1>

Проект содержит следующие этапы:
* Сбор БД и визуализация данных
* **Токенизация, лемматизация и стемминг данных**
* Векторизация
* Классификация
* Подбор гиперпараметров и демонстрация итоговых результатов

Для корректной работы кода необходимо загрузить файлы "ITMO_1.csv" и "ITMO_2.csv", полученные в рамках предыдущего шага проекта.

In [ ]:
# общие библиотеки
import csv
import re
import pandas as pd

# nlp-библиотеки для токенизации
!pip install nltk
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from string import punctuation
from nltk.corpus import stopwords

# для лемматизации
!pip install pymorphy2 > None
import pymorphy2

# для стемминга
from nltk.stem.snowball import RussianStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Токенизация
Токенизация - разделение текста на значимые единицы

In [ ]:
def remove_emojis(text):
    # регулярное выражение для удаления большинства смайликов, которые встречаются в постах
    emoji_pattern = re.compile(r'[^\u0000-\uD7FF\uE000-\uFFFF]', flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
def tokenize_csv(input_file, output_file):
    # предобработка и токенизация текста

    # загрузка стоп-слов на русском языке из nltk
    stop_words = set(nltk.corpus.stopwords.words('russian'))

    # кастомные стоп-слова
    custom_stop_words = {'это', 'вместе', 'а', 'где', 'https', 'с', 'в', 'по', 'далее', 'так', 'быть', 'ru', 'сс', 'сс', '00', '—', '⬇', '✅', '✔', '❗', '⌚'}
    stop_words.update(custom_stop_words)

    # открытие входного csv файла
    with open(input_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        rows = list(reader)

    # создание выходного CSV файла
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        fieldnames = ['text', 'tokenized_text']
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # запись заголовков
        writer.writeheader()

        # обработка каждой строки
        for row in rows:
            text = row['text']

            # удаление смайликов, URL-адресов и символов переноса строки
            text = remove_emojis(text)
            text = re.sub(r'http\S+', '', text)
            text = re.sub(r'\n', ' ', text)

            # токенизация
            tokens = word_tokenize(text.lower())

            # удаление пунктуации и стоп-слов
            punctuations = list(punctuation)
            filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words and token not in punctuations]

            # запись результатов в выходной файл
            writer.writerow({
                'text': text,
                'tokenized_text': ' '.join(filtered_tokens)
            })

# вызов функции
tokenize_csv('ITMO_1.csv', 'ITMO_2.csv')

# Лемматизация
Лемматизация - приведение слова к начальной форме

In [ ]:
def add_lemmatized_column(input_file, output_file):
    # добавление колонки с лемматизацией текста

    #записываем в morph лемматизатор
    morph = pymorphy2.MorphAnalyzer()

    # открываем входной файл
    with open(input_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        rows = list(reader)

    # добавляем новый столбец lemmatized_text
    fieldnames = reader.fieldnames + ['lemmatized_text']

    # записываем в новый столбец данные
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        # обходим каждую строку с токенизированным текстом
        for row in rows:
            text = row['tokenized_text']
            tokens = text.split()
            # лемматизируем текст
            lemmatized_tokens = [morph.parse(token)[0].normal_form for token in tokens]
            lemmatized_text = ' '.join(lemmatized_tokens)
            # непосредственно запись в нужный столбец
            row['lemmatized_text'] = lemmatized_text
            writer.writerow(row)

# вызов функции
add_lemmatized_column('ITMO_2.csv', 'ITMO_2.csv')

# Стемминг
Стемминг - выделение основы слова

In [ ]:
def add_stemmed_column(input_file, output_file):
    # добавление колонки со стеммингом текста

    #записываем в stemmer смемматизатор
    stemmer = RussianStemmer()

    # открываем входной файл
    with open(input_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        rows = list(reader)

    # добавляем новый столбец stemmed_text
    fieldnames = reader.fieldnames + ['stemmed_text']

    # записываем в новый столбец данные
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        # обходим каждую строку с токенизированным текстом
        for row in rows:
            text = row['tokenized_text']
            tokens = text.split()
            # стемминг текста
            stemmed_tokens = [stemmer.stem(token) for token in tokens]
            stemmed_text = ' '.join(stemmed_tokens)
            # непосредственно запись в нужный столбец
            row['stemmed_text'] = stemmed_text
            writer.writerow(row)

# вызов функции
add_stemmed_column('ITMO_2.csv', 'ITMO_2.csv')

In [ ]:
# перенесем столбец popular в новый файл
# открываем исходный CSV-файл для чтения
with open('ITMO_1.csv', 'r', encoding='utf-8') as input_file:
    reader = csv.DictReader(input_file)
    rows = list(reader)

# открываем целевой CSV-файл для чтения
with open('ITMO_2.csv', 'r', encoding='utf-8') as target_file:
    target_reader = csv.DictReader(target_file)
    target_rows = list(target_reader)

# добавляем новый столбец 'popular' в целевой файл
fieldnames = target_reader.fieldnames + ['popular']

# открываем целевой CSV-файл для записи
with open('ITMO_2.csv', 'w', newline='', encoding='utf-8') as output_file:
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    writer.writeheader()

    # обходим каждую строку в целевом файле
    for target_row, source_row in zip(target_rows, rows):
        # копируем значение из столбца 'popular' из исходного файла
        target_row['popular'] = source_row['popular']
        writer.writerow(target_row)

In [ ]:
df = pd.read_csv('ITMO_2.csv')
df

,text,tokenized_text,lemmatized_text,stemmed_text,popular
0,"А что если бы существовал гайд по поступлению,...",существовал гайд поступлению родителей… итмо с...,существовать гайда поступление родителей… итмы...,существова гайд поступлен родителей… итм сбер ...,1
1,NaN,NaN,NaN,NaN,0
2,"Грандиозно, экспертно, футуристично! Вот ка...",грандиозно экспертно футуристично прошел itmo ...,грандиозно экспертный футуристичный пройти itm...,грандиозн экспертн футуристичн прошел itmo con...,0
3,Гоооооооооооо ...на ITMO GO! В начале т...,гоооооооооооо ... itmo go начале тусовки 7 сен...,гоооооооооооо ... itmo go начало тусовка 7 сен...,гооооооооооо ... itmo go начал тусовк 7 сентяб...,1
4,[ ИТМО #KRONBARS] заняла третье место в Чемпио...,итмо kronbars заняла третье место чемпионате р...,итмый kronbars занять третий место чемпионат р...,итм kronbars заня трет мест чемпионат россии-2...,1
...,...,...,...,...,...
15622,Всем привет!,всем привет,весь привет,всем привет,0
15623,и почему тут никто не пишет?,почему никто пишет,почему никто писать,поч никт пишет,0
15624,"ну вот, нас уже трое =)",трое,трое,тро,0
15625,давай пытаться =),давай пытаться,давать пытаться,дава пыта,0


В рамках работы с датасетом было выполнено несколько преобразований. Каждый из них сохранен под определенным порядковым номером:
<li> ITMO_0.csv - исходный датасет в результате парсинга стены vk
<li> ITMO_1.csv - датасет с добавлением столбца популярности поста (если набрал > 30 лайков)
<li> ITMO_2.csv - датасет с токенизацией, лемматизацией и стеммингом
<li> ITMO_2_vectorized_bag_of_words.npz - векторизация через Bag of Words
<li> ITMO_2_vectorized_tf_idf.npz - векторизация через TF-IDF
<li> ITMO_2_vectorized_word2vec.npz.csv - векторизация через Word2Vec
<li> ITMO_3.csv - датасет с добавлением столбца по анализу сентимента (позитив, негатив, нейтралитет)

**Промежуточные выводы:**
<li> Была выполнена токенизация, лемматизация и стемминг текста
<li> При анализе полученных результатов было принято решение, что дальнейшие преобразования будет делать со столбцом с лемматизацией, так как данный подход дал более качественные результаты